In [46]:
import cv2
import numpy as np

* 양방향 필터

In [38]:
src = cv2.imread('./data/lena.bmp', cv2.IMREAD_GRAYSCALE)

data = np.zeros(src.shape, dtype=np.uint8)
noise = cv2.randn(data, mean=0, stddev=10)

src_noise = cv2.add(src, noise)

dst1 = cv2.GaussianBlur(src_noise, ksize=(7, 7), sigmaX=10) #가우시안 필터 잡음은 제거되는데 엣지가 사라짐
dst2 = cv2.bilateralFilter(src_noise, -1, sigmaColor=20, sigmaSpace=10) #10 차이가 나면 놔둔다. 엣지도 꽤나 살림
#sigmaColor을 크게하면 엣지의 기준이 올라가서 엣지가 꽤나 줄어들고 선명도가 살짝 낮아진다.


cv2.imshow('src', src)
cv2.imshow('src_noise', src_noise)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

* 미디언 필터

In [30]:
src = cv2.imread('./data/lena.bmp', cv2.IMREAD_GRAYSCALE)

In [35]:
num = src.shape[0] * src.shape[1] * 0.1 #0.1을 안곱하면 너무 커짐

In [36]:
src.shape

(512, 512)

In [45]:
for i in range(np.int64(num)):
    x = np.random.randint(src.shape[1])
    y = np.random.randint(src.shape[0])
    src[y, x] = (i % 2) * 255 # 0과 1이 반복되어 0과 255로만 집어넣기.
    
gaussian = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=5)
median = cv2.medianBlur(src, ksize=7) #1, 3, 5, 7을 넣어야한다. 홀수여야지  중간 값이 있기에

cv2.imshow('src', src)
cv2.imshow('gaussian', gaussian)
cv2.imshow('median', median)
cv2.waitKey()
cv2.destroyAllWindows()

In [48]:
cv2.destroyAllWindows()

# 모폴로지 연산

* 침식과 팽창

In [49]:
src = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))

erode = cv2.erode(src, kernel, iterations=5)
dilate = cv2.dilate(src, kernel, iterations=5)

cv2.imshow('src', src)
cv2.imshow('erode', erode) #침식 구멍이 커지고 자그만 픽셀 사라짐
cv2.imshow('dilate', dilate) # 팽창 구멍이 사라지지만 자그만 픽셀이 다시생김
cv2.waitKey()
cv2.destroyAllWindows()

* 열기와 닫기

In [ ]:
src = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))

closing = cv2.morphologyEx(src, cv2.MORPH_CLOSE, kernel, iterations=4)
opening = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel, iterations=4)

cv2.imshow('src', src)
cv2.imshow('opening', opening) #열기(축소 팽창) 작은 픽셀 사라짐 구멍 있음
cv2.imshow('closing', closing) #닫기(팽창, 축소)  구멍이 사라짐 작은 픽셀 있음
cv2.waitKey()
cv2.destroyAllWindows()

In [59]:
src = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))

closing = cv2.morphologyEx(src, cv2.MORPH_CLOSE, kernel, iterations=4)
open_close = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=4)

cv2.imshow('src', src)
cv2.imshow('open_close', open_close) #작은픽셀도 사라지고 구멍도 사라짐
cv2.waitKey()
cv2.destroyAllWindows()

# 에지(엣지) 검출과 응용

* 미분 필터

In [110]:
#src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
src = cv2.imread('./data/rect.jpg', cv2.IMREAD_GRAYSCALE)

gx = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3) #소벨마스크 기본마스크 보다 성능이 좋다. 32F는 부동 소수점이다. 1, 0 은 dx, dy이다. x가 1이므로 x의 미분
gy = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3) # x, y가 0, 1이므로 y에 대한 미분.

dstX = cv2.sqrt(np.abs(gx))
dstX = cv2.normalize(dstX, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U) #펼치는 것임 색이 다양해짐. 8U는 uint8임

dstY = cv2.sqrt(np.abs(gy))
dstY = cv2.normalize(dstY, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U) #펼치는 것임 색이 다양해짐. 8U는 uint8임

mag = cv2.magnitude(gx, gy) #벡터 크기 구해줌 화살표의 길이 np.ndarray의 형태이며 0이 최소, 1000정도가 최대
dstM = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

ret, edge = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY) #이진 분류로 만듦 더 뚜렷하게 하려고 임계치가 적당해야지 보기 편함
edge = edge.astype(np.uint8)

cv2.imshow('src', src)
cv2.imshow('dstX', dstX) #X축에 대해서다보니 가로에만 영향
cv2.imshow('dstY', dstY) #Y축에 대해서다보니 세로에만 영향
cv2.imshow('dstM', dstM) # 가로 세로모두에 대해 영향을 받음 뚜렷하게 보인다.
cv2.imshow('edge', edge) #내가 만든 에지
cv2.waitKey()
cv2.destroyAllWindows()
# 가우시안 블러로 노이즈를 조금 제거하면 결과가 더 좋을 듯.

In [98]:
edge.d

dtype('float32')

In [115]:
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
blur = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0.0)

In [120]:
gx1 = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3)
gy1 = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3)
mag1, angle1 = cv2.cartToPolar(gx1, gy1, angleInDegrees=True) #벡터의 크기뿐 아니라 방향도 구해줌

In [121]:
gx2 = cv2.Sobel(blur, cv2.CV_32F, 1, 0, ksize=3)
gy2 = cv2.Sobel(blur, cv2.CV_32F, 0, 1, ksize=3)
mag2, angle2 = cv2.cartToPolar(gx2, gy2, angleInDegrees=True)

In [122]:
ret1, edge1 = cv2.threshold(mag1, 100, 255, cv2.THRESH_BINARY)
edge1 = edge1.astype(np.uint8)

In [123]:
ret2, edge2 = cv2.threshold(mag2, 100, 255, cv2.THRESH_BINARY)
edge2 = edge2.astype(np.uint8)

In [125]:
#cv2.imshow('src', src)
cv2.imshow('edge1', edge1)
cv2.imshow('edge2', edge2) #가우시안 블러를 한 것이 디테일은 부족해도 부드럽고 깔끔함 차이가 확실히 있다. 
cv2.waitKey()
cv2.destroyAllWindows()